In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from openpyxl import load_workbook
from openpyxl.styles import Font
from datetime import datetime

In [2]:
datapath = Path("./data")

In [3]:
df = pd.read_excel(datapath / "hmc_sap_wbs.xlsx")

In [4]:
df.head()

,회사구분,투자구분,프로젝트 정의,Project 명,WBS 요소,WBS 명,시스템 상태,사용자 상태,집행구분,통화,...,추진형태,투자계정,계획계정,예산담당자,연구과제,시작예정일,종료예정일,회계단위,설치장소,자본화일자
0,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0003,시험부품구입비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
1,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0004,공용부품 불출비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
2,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0005,엔진불출비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
3,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0006,연료비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
4,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0007,해외출장비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31


In [5]:
df.columns = df.columns.map(lambda x: x.replace(" ", ""))
df.columns

Index(['회사구분', '투자구분', '프로젝트정의', 'Project명', 'WBS요소', 'WBS명', '시스템상태', '사용자상태',
       '집행구분', '통화', '부대비포함', 'WBS통화키', '청구누적', '최초품의', '최종품의', '전체예산', '전체실적',
       '전체약정', '2025년최초계획', '당년운영금액', '당년가용예산', '2025년계획', '2025년예산',
       '2025년실적', '2025년약정', '2026년이후계획', '2026년이후예산', '2026년이후약정', '책임코스트센터',
       '총괄팀', '통제팀', '투자지역', '레벨텍스트1', '레벨텍스트2', '레벨텍스트3', '레벨텍스트4', '집행유형',
       '추진형태', '투자계정', '계획계정', '예산담당자', '연구과제', '시작예정일', '종료예정일', '회계단위',
       '설치장소', '자본화일자'],
      dtype='object')

In [8]:
df.dropna(subset="프로젝트정의", inplace=True)
df.shape

(1314, 47)

In [16]:
df = df[(df["집행구분"] != "마감") & (df["통화"] != "KRW")]
df.shape

(30, 47)

In [17]:
df["프로젝트정의"].isna().sum()

np.int64(0)

In [18]:
df_real = pd.read_csv(datapath / "manager.txt", delimiter="\t")
df_real.head()

,프로젝트정의,실질담당자
0,P-180116,이중희 책임
1,P-190136,이성학 책임
2,P-220012,이중희 책임
3,P-220214,이중희 책임
4,P-220216,이성학 책임


In [19]:
def analyze_budget(df, mode=None):
    cond = (df["투자구분"] == mode) & (df["당년운영금액"] > 0)
    curr_op_budget = df.loc[cond, ["프로젝트정의", "Project명", "WBS요소", "WBS명", "집행구분", "통화", "당년운영금액", '당년가용예산', 
                                   '2025년실적', '2025년약정', '2026년이후계획']]
    curr_op_budget = pd.merge(curr_op_budget, df_real, on="프로젝트정의", how="left")
    cond = (curr_op_budget["집행구분"] == "미승인") & (curr_op_budget["당년운영금액"] > 0)
    curr_op_budget["미품의"] = curr_op_budget.loc[cond, ["당년운영금액"]]

    cob = curr_op_budget.sum(numeric_only=True)
    total_check = cob['당년가용예산'] + cob['2025년실적'] + cob['2025년약정'] + cob['미품의']
    total_check_info = f"집행 완료: {round((cob['2025년실적'] + cob['2025년약정']) / 1e8, 1)}억, "
    total_check_info += f"품의완/미집행: {round(cob['당년가용예산'] / 1e8, 1)}억, "
    total_check_info += f"미품의: {round(cob['미품의'] / 1e8, 1)}억, "
    total_check_info += f"총계: {round(total_check / 1e8, 1)}억원"
    
    cond = (curr_op_budget["집행구분"] == "미승인") & (curr_op_budget["당년운영금액"] > 0)
    manager_df = curr_op_budget.loc[cond, ["프로젝트정의", "Project명", "실질담당자"]]
    manager_df = manager_df.drop_duplicates(subset="프로젝트정의")
    print(manager_df)
    
    return curr_op_budget, total_check_info, manager_df

In [31]:
def save_to_excel(filename, sheet_name, isfirst=True, curr_op_budget=None, total_check_info=None, manager_df=None):
    if isfirst:
        with pd.ExcelWriter(filename, engine="openpyxl") as writer:
            curr_op_budget.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)
    else:
        with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
            curr_op_budget.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    book = load_workbook(filename=filename)
    sheet = book[sheet_name]
    
    last_row = sheet.max_row
    while last_row > 0 and all(cell.value is None for cell in sheet[last_row]):
        last_row -= 1
    
    col_widths = [10, 50, 12, 42, 8, 8, 12, 12, 12, 12, 12, 12, 12]
    for i, col in enumerate(sheet.columns):
        sheet.column_dimensions[col[0].column_letter].width = col_widths[i]
    
    for col in ["D", "E", "F", "G", "H", "I", "J", "K", "M"]:
        for row in range(2, sheet.max_row + 1):
            cell = sheet[f"{col}{row}"]
            cell.number_format = "#,##0"
    
    sans_font = Font(name="맑은 고딕", size=10, bold=False)
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = sans_font if cell.value is not None else None

    book.save(filename)

In [32]:
time = datetime.now().strftime("%y%m%d")
filename = datapath / f"{time} 예산현황(외화기준).xlsx"

sheet_name = "시설투자(B)"
curr_op_budget, total_check_info, manager_df = analyze_budget(df, mode="B")
save_to_excel(filename, sheet_name, True, curr_op_budget, total_check_info, manager_df)
sheet_name = "기술개발(T)"
curr_op_budget, total_check_info, manager_df = analyze_budget(df, mode="T")
save_to_excel(filename, sheet_name, False, curr_op_budget, total_check_info, manager_df)
sheet_name = "프로젝트(P)"
curr_op_budget, total_check_info, manager_df = analyze_budget(df, mode="P")
save_to_excel(filename, sheet_name, False, curr_op_budget, total_check_info, manager_df)

     프로젝트정의                            Project명   실질담당자
1  R-257419  [장비-신규] 상용디젤엔진 유로7 보쉬 ECU 통신 모듈 구매  이성학 책임
Empty DataFrame
Columns: [프로젝트정의, Project명, 실질담당자]
Index: []
     프로젝트정의                  Project명   실질담당자
0  P-220214  상용차 F-엔진 국내 유로7 법규 대응 개발  이중희 책임
1  P-220216  상용차 G-엔진 국내 유로7 법규 대응 개발  이성학 책임
3  P-220233  상용차 L-엔진 국내 유로7 법규 대응 개발  김주호 책임
6  P-240106        QZc 국6 연비개선 4단계 대응  김주호 책임
